# **SQL QUERY INTO GRANDELECTORS US ELECTION TABLE.**

In [ ]:
# We first load the sql extension into our environment
# This extension will allow us to work with sql on this platform
%load_ext sql
# We will then connect to our in memory sqlite database
%sql sqlite://

'Connected: @None'

In [ ]:
# We will then import the SQL libraries that we will need to use here  

import csv
import pandas as pd

In [ ]:
# Loading the first table from a local CSV file

with open('GrandElectors_by_state.csv','r') as f:
    GrandElectors = pd.read_csv(f, index_col=0, encoding='utf-8') 

# Then storing it in an SQL table of our in memory sqlite database

%sql DROP TABLE if EXISTS GrandElectors;
%sql PERSIST GrandElectors;

 * sqlite://
Done.
 * sqlite://


'Persisted grandelectors'

In [ ]:
# Loading the first table from a local CSV file

with open('Population_by_state.csv','r') as f:
    Population = pd.read_csv(f, index_col=0, encoding='utf-8') 

# Then storing it in an SQL table of our in memory sqlite database

%sql DROP TABLE if EXISTS Population;
%sql PERSIST Population;

 * sqlite://
Done.
 * sqlite://


'Persisted population'

In [ ]:
# Lets preview what we have on our table GrandElectors

%sql select * from GrandElectors LIMIT 9


 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55
Colorado,9
Connecticut,7
District of Columbia,3
Delaware,3


In [ ]:
# Lets preview what we have on our table Population

%sql select * from Population LIMIT 9

 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995
PENNSYLVANIA,12802503
OHIO,11613423
GEORGIA,10214860
NORTH CAROLINA,10042802


***1) To join the 2 tables:***

1.1)  *We First Convert the lowercase to uppercase the values in the Column State*

In [ ]:
# We First Convert the lowercase to uppercase the values in the Column State in the table GrandElectors

%%sql
UPDATE GrandElectors
SET State = UPPER(State);

SELECT * FROM GrandElectors LIMIT 9;

 * sqlite://
51 rows affected.
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55
COLORADO,9
CONNECTICUT,7
DISTRICT OF COLUMBIA,3
DELAWARE,3


1.2) *We now endeavour to Join the two tables.*

In [ ]:
# We now endeavour to Join the two tables.

%%sql
SELECT GrandElectors.State , GrandElectors, Population.Population
FROM GrandElectors INNER JOIN Population ON Population.State = GrandElectors.State
LIMIT 9;

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818
COLORADO,9,5456574
CONNECTICUT,7,3590886
DISTRICT OF COLUMBIA,3,672228
DELAWARE,3,945934


2) ***Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.***

In [ ]:

# This code will change the value of DISTRICT OF COLUMBIA now uppercased within the Column state into DC

%%sql
UPDATE GrandElectors
SET State = "DC"
WHERE State = "DISTRICT OF COLUMBIA";

SELECT * FROM GrandElectors LIMIT 9;

 * sqlite://
1 rows affected.
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55
COLORADO,9
CONNECTICUT,7
DC,3
DELAWARE,3


3) ***To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.***

In [ ]:

# We will make a programs that finds the ratio Grand Electors per capita and name that column Ratio

%%sql  
SELECT *, ( GrandElectors*1.0 / Population*1.0 ) AS Ratio
FROM GrandElectors INNER JOIN Population ON Population.State = GrandElectors.State
LIMIT 9;


 * sqlite://
Done.


State,GrandElectors,State_1,Population,Ratio
ALABAMA,9,ALABAMA,4858979,1.8522409749043987e-06
ALASKA,3,ALASKA,738432,4.0626625065002604e-06
ARIZONA,11,ARIZONA,6828065,1.610998137832607e-06
ARKANSAS,6,ARKANSAS,2978204,2.0146370094190997e-06
CALIFORNIA,55,CALIFORNIA,39144818,1.4050391037710278e-06
COLORADO,9,COLORADO,5456574,1.6493865931260165e-06
CONNECTICUT,7,CONNECTICUT,3590886,1.949379623858847e-06
DELAWARE,3,DELAWARE,945934,3.1714686225466047e-06
FLORIDA,29,FLORIDA,20271272,1.4305959685213637e-06


4) ***To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.***

In [ ]:

# We will write a code that orders the states in decreasing order of GrandElectors per capita Ratio

%%sql  
SELECT *, ( GrandElectors*1.0 / Population*1.0 ) AS Ratio
FROM GrandElectors INNER JOIN Population ON Population.State = GrandElectors.State
ORDER BY Ratio DESC
LIMIT 9;

 * sqlite://
Done.


State,GrandElectors,State_1,Population,Ratio
WYOMING,3,WYOMING,586107,5.118519314732634e-06
VERMONT,3,VERMONT,626042,4.792010759661493e-06
ALASKA,3,ALASKA,738432,4.0626625065002604e-06
NORTH DAKOTA,3,NORTH DAKOTA,756927,3.963394092164766e-06
RHODE ISLAND,4,RHODE ISLAND,1056298,3.7868101615263876e-06
SOUTH DAKOTA,3,SOUTH DAKOTA,858469,3.4945932817609025e-06
DELAWARE,3,DELAWARE,945934,3.1714686225466047e-06
MAINE,4,MAINE,1329328,3.009039153617467e-06
NEW HAMPSHIRE,4,NEW HAMPSHIRE,1330608,3.0061445594795764e-06


5) ***To compute the running total of Grand Electors in that sorted list.***

In [ ]:

# We write a code that gets the running total of all the GrandElectors in the sorted list above

%%sql
SELECT SUM(GrandElectors) , COUNT(GrandElectors)
FROM GrandElectors INNER JOIN Population ON Population.State = GrandElectors.State


 * sqlite://
Done.


SUM(GrandElectors),COUNT(GrandElectors)
535,50


6)***Independently, to compute the half of the total of Grand Electors overall (in the whole country)***

In [ ]:
# We write a code that computes half of the grandElectors Overall in the whole country

%%sql
SELECT SUM(GrandElectors)/2, COUNT(GrandElectors) FROM GrandElectors


 * sqlite://
Done.


SUM(GrandElectors)/2,COUNT(GrandElectors)
269,51


7) ***To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.***

In [ ]:

# We write a code that filters the sorted list of states in order to keep the top ones enabling us to reach the computed threshold.
# This ends when the SUM(GrandElectors) is >= 269 needed to win an election.
# These are the states that will be prioritized to win the election.
# Though i calculated it manually, sad

%%sql  
SELECT *, ( GrandElectors*1.0 / Population*1.0 ) AS Ratio
FROM GrandElectors INNER JOIN Population ON Population.State = GrandElectors.State
ORDER BY Ratio DESC
limit 12


 * sqlite://
Done.


State,GrandElectors,State_1,Population,Ratio
WYOMING,3,WYOMING,586107,5.118519314732634e-06
VERMONT,3,VERMONT,626042,4.792010759661493e-06
ALASKA,3,ALASKA,738432,4.0626625065002604e-06
NORTH DAKOTA,3,NORTH DAKOTA,756927,3.963394092164766e-06
RHODE ISLAND,4,RHODE ISLAND,1056298,3.7868101615263876e-06
SOUTH DAKOTA,3,SOUTH DAKOTA,858469,3.4945932817609025e-06
DELAWARE,3,DELAWARE,945934,3.1714686225466047e-06
MAINE,4,MAINE,1329328,3.009039153617467e-06
NEW HAMPSHIRE,4,NEW HAMPSHIRE,1330608,3.0061445594795764e-06
MONTANA,3,MONTANA,1032949,2.904306020916812e-06
